# Fine tuning your function for Function-calling
https://huggingface.co/learn/agents-course/bonus-unit1/introduction

In [1]:
# In a typical conversation you would interact in the following way
conversation = [
    {"role": "user", "content": "I need help with my order"},
    {"role": "assistant", "content": "I'd be happy to help. Could you provide your order number?"},
    {"role": "user", "content": "It's ORDER-123"},
]

In [ ]:
# Function calling brings in a new role for Action and one new role for Observation
conversation = [
    {
        "role": "user",
        "content": "What's the status of my transaction T1001?"
    },
    {
        "role": "assistant",
        "content": "",
        "function_call": {
            "name": "retrieve_payment_status",
            "arguments": "{\"transaction_id\": \"T1001\"}"
        }
    },
    {
        "role": "tool",
        "name": "retrieve_payment_status",
        "content": "{\"status\": \"Paid\"}"
    },
    {
        "role": "assistant",
        "content": "Your transaction T1001 has been successfully paid."
    }
]

# Fine-Tuning a model for Function-Calling
https://huggingface.co/agents-course/notebooks/blob/main/bonus-unit1/bonus-unit1.ipynb

In [21]:
# Install transformers
%pip install transformers

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.3.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [22]:
# %pip install -q -U bitsandbytes # Efficient memory management and operations on large-scale neural networks
# %pip install -q -U peft # fine-tuning models with fewer parameters
# %pip install -q -U trl # Reinforcement Learning Libreries for implementing learning algorithms.
# %pip install -q -U tensorboardX # Visualize pytorch training results in a TensorBoard
# %pip install -q wandb #Weights and Biases tool for tracking experiments, visualizing results, and managing collaberation.
# %pip install -q -U python-dotenv

In [ ]:
# Import necessary libraries
from enum import Enum  # Enum for creating enumerations
from functools import partial  # Partial function application
import pandas as pd  # Pandas for data manipulation
import torch  # PyTorch for deep learning
import json  # JSON module for handling JSON data

# Import Hugging Face Transformers and related libraries
from transformers import AutoModelForCausalLM, AutoTokenizer, set_seed  # Transformer model, tokenizer, and seed setting
from datasets import load_dataset  # Load datasets from Hugging Face's datasets library
from trl import SFTConfig, SFTTrainer  # TRL (Transformers Reinforcement Learning) configurations and trainer
from peft import LoraConfig, TaskType  # Parameter-efficient fine-tuning (PEFT) configurations

# Load environment variables from a .env file
from dotenv import load_dotenv  
import os
from huggingface_hub import login

load_dotenv()  # Load environment variables from the .env file

# Retrieve the token from environment variables
hf_token = os.getenv("HF_TOKEN")

# Authenticate hugging face token
if hf_token:
    login(token=hf_token)
else:
    print("Hugging Face token not found.")

seed = 42 # from the guide
set_seed(seed)  # Set the seed for reproducibility

import os
os.environ

"{{ bos_token }}
{% if messages[0]['role'] == 'system' %}
{{ raise_exception('System role not supported') }}
{% endif %}
{% for message in messages %}
{{ '<start_of_turn>' + message['role'] + '\n' + message['content'] | trim + '<end_of_turn><eos>\n' }}
{% endfor %}
{% if add_generation_prompt %}
{{'<start_of_turn>model\n'}}
{% endif %}"

In [34]:
# %pip install -q -U tabulate
import tabulate 
model_name = "google/gemma-2-2b-it"
dataset_name = "Jofthomas/hermes-function-calling-thinking-V1"
print(dataset_name)

tokenizer = AutoTokenizer.from_pretrained(model_name)

tokenizer.chat_template = "{{ bos_token }}{% if messages[0]['role'] == 'system' %}{{ raise_exception('System role not supported') }}{% endif %}{% for message in messages %}{{ '<start_of_turn>' + message['role'] + '\n' + message['content'] | trim + '<end_of_turn><eos>\n' }}{% endfor %}{% if add_generation_prompt %}{{'<start_of_turn>model\n'}}{% endif %}"

def preprocess(sample):
   messages = sample["messages"]
   first_message = messages[0] 

   # Instead of adding a system message, we merge the content into the first use message
   if first_message["role"] == "system":
      system_message_content = first_message["content"]
      # Merge system content with the first user message
      messages[1]["content"] = system_message_content + "Also, before making a call to a function take the time to plan the function to take. Make that thinking process between <think>{your thoughts}</think>\n\n" + messages[1]["content"]
      # Remove the system message form the conversation
      messages.pop(0)

   return {"text": tokenizer.apply_chat_template(messages, tokenize=False)}

dataset = load_dataset(dataset_name)
dataset = dataset.rename_column("conversations", "messages")

Jofthomas/hermes-function-calling-thinking-V1


In [35]:
dataset = dataset.map(preprocess, remove_columns="messages")
dataset = dataset["train"].train_test_split(0.1)
print(dataset)

Map: 100%|██████████| 3570/3570 [00:00<00:00, 7349.59 examples/s]

DatasetDict({
    train: Dataset({
        features: ['text'],
        num_rows: 3213
    })
    test: Dataset({
        features: ['text'],
        num_rows: 357
    })
})


In [36]:
print(dataset["train"][8]["text"])
# print(dataset)

<bos><start_of_turn>human
You are a function calling AI model. You are provided with function signatures within <tools></tools> XML tags.You may call one or more functions to assist with the user query. Don't make assumptions about what values to plug into functions.Here are the available tools:<tools> [{'type': 'function', 'function': {'name': 'search_movies', 'description': 'Search for movies based on given criteria', 'parameters': {'type': 'object', 'properties': {'genre': {'type': 'string', 'description': 'The genre of the movie'}, 'year': {'type': 'integer', 'description': 'The year of release'}, 'rating': {'type': 'number', 'description': 'The minimum rating of the movie'}}, 'required': []}}}, {'type': 'function', 'function': {'name': 'convert_currency', 'description': 'Convert currency from one unit to another', 'parameters': {'type': 'object', 'properties': {'amount': {'type': 'number', 'description': 'The amount to convert'}, 'from_currency': {'type': 'string', 'description': 

In [31]:
# Sanity check
print(tokenizer.pad_token)
print(tokenizer.eos_token)

<pad>
<eos>


In [ ]:
class ChatmlSpecialTokens(str, Enum):
    tools = "<tools>"
    eotools = "</tools>"
    think = "<think>"
    eothink = "</think>"
    tool_call = "<tool_call>"
    eottool_call = "</tool_call>"
    tool_response = "<tool_response>"
    eotool_response = "</tool_response>"
    pad_token = "<pad>"
    eos_token = "<eos>"
    @classmethod
    def list(cls):
        return [c.value for c in cls]

tokenizer = AutoTokenizer.from_pretrained(
    pretrained_model_name_or_path=model_name,
    pad_token=ChatmlSpecialTokens.pad_token.value,
    additional_special_tokens=ChatmlSpecialTokens.list(),
)
tokenizer.chat_template = "{{ bos_token }}{% if messages[0]['role'] == 'system' %}{{ raise_exception('System role not supported') }}{% endif %}{% for message in messages %}{{ '<start_of_turn>' + message['role'] + '\n' + message['content'] | trim + '<end_of_turn><eos>\n' }}{% endfor %}{% if add_generation_prompt %}{{'<start_of_turn>model\n'}}{% endif %}"

model = AutoModelForCausalLM.from_pretrained(
    pretrained_model_name_or_path=model_name,
    attn_implmentation='eager',
    device_map="auto")
print(f"Number of tokens: {len(tokenizer)}")
model.resize_token_embeddings(len(tokenizer))
model.to(torch.bfloat16)